In [1]:
import configparser
from datetime import datetime
import os
from pyspark.sql import SparkSession
from pyspark.sql import functions as F
from pyspark.sql.functions import udf, col
from pyspark.sql.functions import year, month, dayofmonth, hour, weekofyear, date_format

import os
import glob
import shutil

# import pandas as pd
# pd.set_option('display.max_rows', 500)
# pd.set_option('display.max_columns', 500)
# pd.set_option('display.width', 1000)

VBox()

Starting Spark application


ID,YARN Application ID,Kind,State,Spark UI,Driver log,Current session?
3,application_1671988735950_0004,pyspark,idle,Link,Link,✔


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

SparkSession available as 'spark'.


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [2]:
def create_spark_session():
    spark = SparkSession \
        .builder \
        .config("spark.jars.packages", "org.apache.hadoop:hadoop-aws:2.7.0") \
        .getOrCreate()
    return spark

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [3]:
spark = SparkSession.builder.getOrCreate()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [4]:
input_data = "s3a://udacity-dend/"
output_data = "s3a://spark-project-output-4/"

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [5]:
def process_song_data(spark, input_data, output_data):
    # get filepath to song data file
    song_data = input_data + "/song_data/*/*/*/*"
    
    # read song data file
    df = spark.read.json(song_data)

    # extract columns to create songs table
    songs_table = (
        df
        .select(
            "song_id",
            "title",
            "artist_id",
            "year",
            "duration",
        )
        .distinct()
    )
    
    # write songs table to parquet files partitioned by year and artist
    songs_table.write.parquet(output_data + "songs.parquet")

    # extract columns to create artists table
    artists_table = (
        df
        .select(
            "artist_id",
            F.col("artist_name").alias("name"),
            F.col("artist_location").alias("location"),
            F.col("artist_latitude").alias("latitude"),
            F.col("artist_longitude").alias("longitude"),
        )
        .distinct()
    )
    
    # write artists table to parquet files
    artists_table.write.parquet(output_data + "artists.parquet")
    
    return songs_table



df_songs = process_song_data(spark=spark, input_data=input_data, output_data="")
df_songs.limit(10).show()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

'path hdfs://ip-172-31-22-55.ec2.internal:8020/user/livy/songs.parquet already exists.;'
Traceback (most recent call last):
  File "<stdin>", line 22, in process_song_data
  File "/usr/lib/spark/python/lib/pyspark.zip/pyspark/sql/readwriter.py", line 841, in parquet
    self._jwrite.parquet(path)
  File "/usr/lib/spark/python/lib/py4j-0.10.7-src.zip/py4j/java_gateway.py", line 1257, in __call__
    answer, self.gateway_client, self.target_id, self.name)
  File "/usr/lib/spark/python/lib/pyspark.zip/pyspark/sql/utils.py", line 69, in deco
    raise AnalysisException(s.split(': ', 1)[1], stackTrace)
pyspark.sql.utils.AnalysisException: 'path hdfs://ip-172-31-22-55.ec2.internal:8020/user/livy/songs.parquet already exists.;'



In [6]:
df_songs.printSchema()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

name 'df_songs' is not defined
Traceback (most recent call last):
NameError: name 'df_songs' is not defined



In [7]:
df_songs.count()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

name 'df_songs' is not defined
Traceback (most recent call last):
NameError: name 'df_songs' is not defined



In [8]:
def process_log_data(spark, input_data, output_data):
    # get filepath to log data file
    log_data = input_data + "log_data/*/*"

    # read log data file
    df = spark.read.json(log_data)
    
    # # filter by actions for song plays
    df = df.filter(F.col("page") == "NextSong")

    # extract columns for users table
    df = (
        df
        .withColumnRenamed("userId", "user_id")
        .withColumnRenamed("firstName", "first_name")
        .withColumnRenamed("lastName", "last_name")
    )
    users_table = (
        df
        .select(
            "user_id",
            "first_name",
            "last_name",
            "gender",
            "level"
        )
        .distinct()
    )

    # write users table to parquet files
    users_table.write.parquet(output_data + "users.parquet")

    # create timestamp column from original timestamp column
    df = df.withColumn("start_time", F.to_timestamp(F.col("ts")/1000))
    
    # # extract columns to create time table
    time_table = (
        df
        .withColumn("hour", F.hour("start_time"))
        .withColumn("day", F.dayofmonth("start_time"))
        .withColumn("week", F.weekofyear("start_time"))
        .withColumn("year", F.year("start_time"))
        .withColumn("weekday", (F.dayofweek("start_time") + 6) % 7)
        .select(
            "start_time",
            "hour",
            "day",
            "week",
            "year",
            "weekday",
        )
        .distinct()
    )
    
    # write time table to parquet files partitioned by year and month
    time_table.write.parquet(output_data + "time.parquet")

    # read in song data to use for songplays table
    song_df = spark.read.parquet(output_data + "songs.parquet")

    # extract columns from joined song and log datasets to create songplays table 
    songplays_table = (
        df
        .join(
            song_df,
            [
                df["song"] == song_df["title"],
                df["length"] == song_df["duration"]
            ],
            "left"
        )
        .withColumnRenamed("sessionId", "session_id")
        .withColumnRenamed("userAgent", "user_agent")
        .withColumn("songplay_id", F.concat_ws("-", F.col("session_id"), F.col("ts").cast("string")))
        .select(
            "songplay_id",
            "start_time",
            "user_id",
            "level",
            "song_id",
            "artist_id",
            "session_id",
            "location",
            "user_agent"
        )
    )

    # write songplays table to parquet files partitioned by year and month
    songplays_table.write.parquet(output_data + "songplays.parquet")
    
    return songplays_table


VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [9]:
df = process_log_data(spark=spark, input_data=input_data, output_data=output_data)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

'Path does not exist: s3a://spark-project-output-4/songs.parquet;'
Traceback (most recent call last):
  File "<stdin>", line 59, in process_log_data
  File "/usr/lib/spark/python/lib/pyspark.zip/pyspark/sql/readwriter.py", line 316, in parquet
    return self._df(self._jreader.parquet(_to_seq(self._spark._sc, paths)))
  File "/usr/lib/spark/python/lib/py4j-0.10.7-src.zip/py4j/java_gateway.py", line 1257, in __call__
    answer, self.gateway_client, self.target_id, self.name)
  File "/usr/lib/spark/python/lib/pyspark.zip/pyspark/sql/utils.py", line 69, in deco
    raise AnalysisException(s.split(': ', 1)[1], stackTrace)
pyspark.sql.utils.AnalysisException: 'Path does not exist: s3a://spark-project-output-4/songs.parquet;'



In [10]:
df.count()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

name 'df' is not defined
Traceback (most recent call last):
NameError: name 'df' is not defined

